NBA Stats to Salary

Team Wizards

Trent Thompson (trentthompson)
Alexander Peal (alexanderpeal)
Dylan Grim (dylang04)
Pranav Ramasubbiah Rajavel (pranavrajavel)

Our project will compare the salary of various NBA player throughout the organization and compare their salary to their stats. This way you can find the amount of points per dollar they are paid and the same with reboundss assists steals and blocks. There can also be a formula made weighing values of each stat to find the best contracts in the league and maybe seeing what team has the best contracts and comparing that to whether they are good or not. 

Who has the most value players?
Who is the highest value player in the league?
What is the weight of each stat? One point should not equal one block.

The website we will use for the stats of the players is: https://www.basketball-reference.com/ They have all the stats and salaries of the players in the league. 